In [1]:
import grid2op
import numpy as np
import torch

In [2]:
env = grid2op.make("l2rpn_neurips_2020_track2_small")

In [3]:
from grid2op.Converter import IdToAct

In [4]:
action_space = env.action_space

In [7]:
-np.prod(action_space.shape).item()

-609357268728

In [14]:
env.unwrapped.id

AttributeError: 'Environment_l2rpn_neurips_2020_track2_x1' object has no attribute 'unwrapped'

In [13]:
env.chronics_handler.real_data().ipynb_checkp

TypeError: 'NoneType' object is not callable

In [92]:
change_line = action_space.get_change_line_status_vect()
set_line = action_space.get_set_line_status_vect()

In [93]:
change_line[0] = True
change_line[1] = True
set_line[2] = 1

In [94]:
act = action_space({"change_line_status": change_line,
                    "set_line_status": set_line,
                    "set_bus": {"lines_ex_id": [(18,2)], "generators_id": [(3,2), (4,2)]},
                    "redispatch": [(0,+10)]})

In [95]:
print(act)

This action will:
	 - NOT change anything to the injections
	 - Redispatch gen_3_0 of 10.0
	 - force reconnection of 1 powerlines ([2])
	 - switch status of 2 powerlines ([0 1])
	 - NOT switch anything in the topology
	 - Set the bus of the following element:
	 	 - assign bus 2 to generator 3 [on substation 9]
	 	 - assign bus 2 to generator 4 [on substation 10]
	 	 - assign bus 2 to line (extremity) 18 [on substation 79]


In [99]:
len(IdToAct.get_all_unitary_redispatch(action_space))

256

In [43]:
env.n_gen + env.n_load + env.n_line * 2

533

In [33]:
env.n_line * 2 + 533 * 2 + env.n_gen

1500

In [54]:
act_dict = act.impact_on_objects()

In [55]:
act_dict

{'has_impact': True,
 'injection': {'changed': False, 'count': 0, 'impacted': []},
 'force_line': {'changed': True,
  'reconnections': {'count': 1, 'powerlines': array([2])},
  'disconnections': {'count': 0, 'powerlines': []}},
 'switch_line': {'changed': True, 'count': 2, 'powerlines': array([0, 1])},
 'topology': {'changed': True,
  'bus_switch': [],
  'assigned_bus': [{'bus': 2,
    'object_type': 'generator',
    'object_id': 3,
    'substation': 9},
   {'bus': 2, 'object_type': 'generator', 'object_id': 4, 'substation': 10},
   {'bus': 2,
    'object_type': 'line (extremity)',
    'object_id': 18,
    'substation': 79}],
  'disconnect_bus': []},
 'redispatch': {'changed': False, 'generators': []}}

In [83]:
{s['substation'] for s in act_dict["topology"]["assigned_bus"]}

{9, 10, 79}

In [57]:
len(act_dict["topology"]["assigned_bus"])

3

In [58]:
len(act_dict["topology"]["disconnect_bus"])

0

In [59]:
len(act_dict["redispatch"]["generators"])

0

In [66]:
env.load_to_subid

array([  0,   1,   2,   3,   5,   6,   7,  10,  11,  12,  13,  14,  15,
        16,  17,  18,  19,  20,  21,  22,  23,  26,  27,  28,  30,  31,
        32,  33,  34,  35,  38,  39,  40,  41,  42,  43,  44,  45,  46,
        47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  57,  58,  59,
        61,  65,  66,  69,  71,  72,  73,  74,  75,  76,  77,  78,  79,
        81,  82,  83,  84,  85,  87,  89,  90,  91,  92,  93,  94,  95,
        96,  97,  98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108,
       109, 111, 112, 113, 114, 115, 116, 117], dtype=int32)

In [69]:
env.line_ex_to_subid

array([  1,   2,  11,  70,  71,  71,  72,  73,  74,  74,  74,  76,  76,
        11,  76,  77,  78,  79,  79,  79,  81,  82,  83,  84,  11,  84,
        85,  87,  88,  88,  89,  89,  90,  91,  91,  11,  91,  92,  93,
        93,  94,  95,  95,  95,  96,  97,  12,  98,  99,  99,  95,  96,
        99,  99, 100, 101, 101,  13, 102, 103, 103, 104, 105, 104, 105,
       106, 107, 106,  14, 108, 109, 109, 110, 111, 112, 112, 113, 114,
       114,  14, 116, 117, 117,  15,  16,   4,  16,  17,  18,  19,  18,
        20,  21,  22,  23,  24,   4,  26,  27,  28,  29,  29,  30,  30,
        31,  31,  31,   5,  32,  33,  35,  36,  36,  35,  36,  38,  39,
        37,   6,  39,  40,  41,  41,  43,  42,  44,  45,  46,  47,   8,
        48,  48,  48,  48,  48,  49,  50,  51,  52,  53,   9,  53,  53,
        54,  55,  55,  56,  56,  57,  57,  58,  10,  58,  58,  58,  59,
        60,  60,  61,  61,  63,  64,  10,  64,  65,  65,  65,  66,  66,
        68,  68,  69,  69,   4,  24,  79,  85,  67,  16,  36,  5

In [70]:
env.line_or_to_subid

array([  0,   0,  10,  69,  23,  70,  70,  69,  69,  68,  73,  75,  68,
         1,  74,  76,  77,  76,  76,  78,  76,  81,  82,  82,   2,  83,
        84,  84,  84,  87,  88,  88,  89,  88,  88,   6,  90,  91,  91,
        92,  93,  79,  81,  93,  79,  79,  10,  79,  91,  93,  94,  95,
        97,  98,  99,  91, 100,  11,  99,  99, 102, 102,  99, 103, 104,
       104, 104, 105,  12, 107, 102, 108, 109, 109,  16,  31,  31,  26,
       113,  13,  11,  74,  75,  11,  14,   3,  15,  16,  17,  18,  14,
        19,  20,  21,  22,  22,   2,  24,  26,  27,   7,  25,  16,  28,
        22,  30,  26,   4,  14,  18,  34,  34,  32,  33,  33,  36,  36,
        29,   5,  38,  39,  39,  40,  42,  33,  43,  44,  45,  45,   7,
        46,  41,  41,  44,  47,  48,  48,  50,  51,  52,   8,  48,  48,
        53,  53,  54,  55,  49,  55,  50,  53,   3,  55,  55,  54,  58,
        58,  59,  59,  60,  62,  37,   4,  63,  48,  48,  61,  61,  65,
        46,  48,  68,  23,   7,  25,  80,  86, 115,  29,  37,  6

In [74]:
"day_of_week,hour_of_day,minute_of_hour,prod_p,prod_v,load_p,load_q,actual_dispatch,target_dispatch,topo_vect,time_before_cooldown_line,time_before_cooldown_sub,rho,timestep_overflow,line_status".split(",")

['day_of_week',
 'hour_of_day',
 'minute_of_hour',
 'prod_p',
 'prod_v',
 'load_p',
 'load_q',
 'actual_dispatch',
 'target_dispatch',
 'topo_vect',
 'time_before_cooldown_line',
 'time_before_cooldown_sub',
 'rho',
 'timestep_overflow',
 'line_status']

In [86]:
{0, 1, 2} | {1,2, 3} | {2, 3, 4}

{0, 1, 2, 3, 4}

In [21]:
a = np.random.rand(3, 5)

In [22]:
a = torch.tensor(a).to(torch.float32)

In [23]:
act = torch.tensor([1, 3, 4])

In [24]:
reward = torch.tensor([10, 100, 1000]).to(torch.float32)

In [25]:
a[np.arange(3), act] = reward

In [26]:
a

tensor([[2.5946e-01, 1.0000e+01, 8.6356e-01, 9.7282e-01, 7.8723e-01],
        [6.7409e-01, 8.3620e-01, 3.8430e-01, 1.0000e+02, 6.7727e-01],
        [1.6835e-01, 9.4377e-01, 8.8787e-01, 4.9811e-01, 1.0000e+03]])